# Benchmark

Import the necessary libraries and the queries. 

In [ ]:
import os
import json
from evaluation.benchmarks import Benchmarks
from whoosh.scoring import BM25F
from doc2vec.doc2vec_model import Doc2VecModel
from sentiment.sentiment_model import SentimentWeightingModel, AdvancedSentimentWeightingModel
from index import Index
from model import IRModel

# file containing benchmark queries
file_path = os.path.join("evaluation", "queries.json")

with open(file_path) as f:
    queries = json.load(f)

# Models that need to be tested. 
models = [
	(BM25F(), "BM25F"),
 	#(Doc2VecModel(), "Doc2Vec"),
	(SentimentWeightingModel(), "Base Sentiment"),
	(AdvancedSentimentWeightingModel(), "Advanced Sentiment" ) 
]

In [ ]:
indexes = [i for i in range(len(b.queries))]
print("Benchmark queries' indexes:")
print(' '.join(map(str, indexes)))

In [ ]:
try:
    examined_q = 0
    print("User Information Need: " + b.queries[examined_q]["UIN"])
except IndexError as e:
    print(e)
    print("index not valid")

In [ ]:
import seaborn as sns
import pandas as pd

b = Benchmark(examined_q)